In [ ]:
# notebook to generate stuff for apeval, I'm assuming just load then save to dset? 
# maybe 
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from datasets import Dataset
import sys
sys.path.append("../../src/")
import json

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils.data.prompt_utils import splitter

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("../../outputs/models/ultra/tiny_dpo_tulu/")
inpmod = AutoModelForCausalLM.from_pretrained("../../outputs/models/ultra/tiny_dpo_tulu/", device_map=0)

In [ ]:
tokenizer.padding_size="left"
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
def tulu_pf(question, answer):
    return "<user>\n"+question+"\n<assistant>\n"+answer

In [ ]:
generation_kwargs = {
    "min_length": -1,
    "max_new_tokens":256,
    #"top_k": 0.0,
    "top_p": 0.9,
    "temperature": 0.9,
    "do_sample": True,
    "repetition_penalty": 1.2,
    "pad_token_id": tokenizer.pad_token_id,
    "eos_token_id": tokenizer.eos_token_id,
}

In [ ]:
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]
# for example in eval_set:
#     # generate here is a placeholder for your models generations
#     inp = 
#     example["output"] = inpmod(example["instruction"], )
#     example['generator'] = "newmodel"

In [27]:
tmpouts = pd.read_json("../../outputs/results/genouts/eurusrm/ppo_eurusrmbaseline275.jsonl", orient='records', lines=True)
len(tmpouts)

804

In [28]:
name = 'eurusactivemain675'
name = 'eurusbase275'

results = []
for i, row in tmpouts.iterrows():
    q, a = splitter(row['response'])
    results.append({
        'instruction':q.strip(),
        'generator':name,
        'output':a.strip()
    })
    assert len(a)>0

In [29]:
with open('../../'+name+'.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [24]:
results[4]

{'instruction': 'How do I wrap a present neatly?',
 'generator': 'eurusactivemain675',
 'output': 'To wrap a present neatly, start by selecting wrapping paper that coordinates with the gift and your overall theme or color scheme. Fold the paper in half lengthwise and place it over the top of the present, making sure to tuck any excess around the edges. Then, fold down each corner diagonally towards the center, and secure them using ribbon, twine, or masking tape. Next, use scissors or a craft knife to cut off any remaining paper flaps along the bottom edge, ensuring they are even on both sides before taping or gluing them shut for added security. Finally, add a bow or other decorative elements like stickers or stamps for some extra flair! With these steps, you can create an attractive and practical package that will make the perfect gift for anyone special in your life. Happy wrapping!'}

In [ ]:
prompt = [tulu_pf(eval_set['instruction'][i], "") for i in range(4)] 
inps = tokenizer(prompt, return_tensors="pt", padding=True).to(inpmod.device)
output = tokenizer.batch_decode(inpmod.generate(**inps, **generation_kwargs), skip_special_tokens=True)

In [ ]:
def row_rev_tulu(rw, fv="ppo_conf_active_newalgoultra75"):
    tmp = {'instruction': rw['question'][len("<user>\n"):-len("\n<assistant>\n")], 'generator':fval+"v2"}
    tmp['output'] = rw['response'][len(rw['question']):]
    return tmp

In [ ]:
fval = "golddpo25"
fname = "../../outputs/results/genouts/ultra/"+fval+".jsonl"
tmpdf = pd.read_json(fname, orient='records', lines=True)
results = []
for i, row in tmpdf.iterrows():
    results.append(row_rev_tulu(row, fval))
newdata = Dataset.from_list(results)
newdata[0]

In [ ]:
# newdata.select(range(10)).to_json("../../testapeval.json")
with open('../../'+fval+'v2.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [ ]:
row_rev_tulu(tmpdf.loc[0])